<a href="https://colab.research.google.com/github/good-thinking/easy-coding2/blob/main/class2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error 
from math import sqrt
import matplotlib.pyplot as plt

from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet


from scipy.stats.distributions import chi_gen

from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.svm import SVR
from sklearn.linear_model import BayesianRidge

from sklearn.neural_network import MLPRegressor

import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.metrics import Accuracy

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor


In [ ]:
class numeric :
  def __init__(self):
    self.df = 0
    self.data_IQR2 = 0
    self.train_test_data = 0
    self.Y_value = 0
    self.Y_test = 0
  
  def read_file(self, file_address) :
    print("file read")
    self.df = pd.read_csv(file_address, encoding = "cp949")
  
  def normalization(self, column_number):
#sns.distplot(self.df[self.df.columns[column_number]])
    print("normalization")
    self.df[self.df.columns[column_number]] = np.log1p(self.df[self.df.columns[column_number]])
    sns.distplot(self.df[self.df.columns[column_number]])

  def data_IQR(self, column_number) :
    print("-------------- Data_IQR ---------------")
    q1 = self.df[self.df.columns[column_number]].quantile(q=0.25)
    q3 = self.df[self.df.columns[column_number]].quantile(q=0.75)
    IQR = q3-q1
    self.data_IQR2 = self.df[(self.df[self.df.columns[column_number]] < q3 + IQR * 1.5) & 
                 (self.df[self.df.columns[column_number]] > q1 - IQR * 1.5)]
    sns.distplot(self.df[self.df.columns[column_number]])
    self.df = self.data_IQR2

  def before_making_model(self, X_list,Y_list,column_number):
    num = X_list
    cg = Y_list
    X = self.df[num+cg]
    Y = self.df[self.df.columns[column_number]]
    ct = ColumnTransformer([("scaling", StandardScaler(), num), 
                        ("onehot", OneHotEncoder(sparse = False), cg)])
    X_train, X_test, Y_train, self.Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
   
    ct.fit(X_train)
    X_train = ct.transform(X_train)
    X_test = ct.transform(X_test)
    self.train_test_data = [X_train, X_test, Y_train, self.Y_test]
  
  def before_making_model2(self, X_list,Y_list,column_number,confirm):
    num = X_list
    cg = Y_list
    X = self.df[num+cg]
    Y = self.df[self.df.columns[column_number]]
    X_train, X_test, Y_train, self.Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
    self.train_test_data = [X_train, X_test, Y_train, self.Y_test]

  def linearRegress(self, train_test_data,columns):
    print("----------LinearRegression value-----------")
    X_train, X_test, Y_train, self.Y_test = self.train_test_data
    lr = LinearRegression().fit(X_train, Y_train) 
    Y_pred = lr.predict(X_test)
    print("predict value\n",Y_pred)
    print("[linear]")
    print("train R2 : {:.3f}".format(lr.score(X_train, Y_train)))
    print("test R2 : {:.3f}".format(lr.score(X_test, self.Y_test))) 
    rmse = sqrt(mean_squared_error(self.Y_test, Y_pred)) 
    print("RMSE : {:.3f}". format(rmse)) 
    print(columns)
    print("절편", np.round(lr.intercept_, 3))
    print("가중치(beta)", np.round(lr.coef_, 3))
    self.Y_value = [Y_pred, self.Y_test]

  def Reset_index(self, Y_test,column_number):
    self.Y_test = (pd.DataFrame(self.Y_test)).reset_index()
    self.Y_test = pd.DataFrame(self.Y_test[self.Y_test.columns[column_number]])
  
  def plot_picture(self, Y_pred,Y_test):
    print("----------- plot grow----------- ")
    %matplotlib Inline
    plt.plot(Y_pred)
    plt.plot(self.Y_test)

  def Ridge_function(self,train_test_data):
    X_train, X_test, Y_train, self.Y_test = self.train_test_data
    lr_ridge = Ridge(random_state = 0).fit(X_train, Y_train)
    Y_pred = lr_ridge.predict(X_test)
    rmse = sqrt(mean_squared_error(self.Y_test, Y_pred))
    print("[ridge]")
    print("ridge R2 : {:.3f}".format(lr_ridge.score(X_train, Y_train)))
    print("RMSE : {:.3f}".format(rmse))
    print("절편", np.round(lr_ridge.intercept_, 3))
    print("가중치(beta)", np.round(lr_ridge.coef_, 3))
    self.Y_value = [Y_pred, self.Y_test]

  def Lasso_function(self, train_test_data):
    X_train, X_test, Y_train, self.Y_test = self.train_test_data
    lr_lasso = Lasso(random_state = 0, alpha = 0.01, max_iter = 1000).fit(X_train, Y_train)
    Y_pred = lr_lasso.predict(X_test)

    rmse = sqrt(mean_squared_error(self.Y_test, Y_pred))
    print("[lasso]")
    print("lasso R2 : {:.3f}".format(lr_lasso.score(X_train, Y_train)))
    print("RMSE : {:.3f}".format(rmse))
    print("절편", np.round(lr_lasso.intercept_, 3))
    print("가중치(beta)", np.round(lr_lasso.coef_, 3))
    self.Y_value = [Y_pred, self.Y_test]

  def Elastic_function(self,train_test_data):
    X_train, X_test, Y_train, self.Y_test = self.train_test_data
    lr_ela = ElasticNet(alpha = 0.01).fit(X_train, Y_train)
    Y_pred = lr_ela.predict(X_test)

    rmse = sqrt(mean_squared_error(self.Y_test, Y_pred))
    print("[elastic]")
    print("ela R2 : {:.3f}".format(lr_ela.score(X_train, Y_train)))
    print("RMSE : {:.3f}".format(rmse))
    print("절편", np.round(lr_ela.intercept_, 3))
    print("가중치(beta)", np.round(lr_ela.coef_, 3))
    self.Y_value = [Y_pred, self.Y_test]
  
  def knn_reg_function(self,train_test_data):
    X_train, X_test, Y_train, self.Y_test = self.train_test_data
    knn_reg = KNeighborsRegressor(n_neighbors=9, p =2)
    knn_reg.fit(X_train, Y_train)
    Y_pred = knn_reg.predict(X_test)
    print("[knn neighbors]")
    print("accuracy: {:.3f}".format(knn_reg.score(X_train, Y_train)))
    rmse = sqrt(mean_squared_error(self.Y_test, Y_pred))
    print("RMSE: {:.3f}".format(rmse))
    self.Y_value = [Y_pred, self.Y_test]

  def decision_tree(self, train_test_data):
    X_train, X_test, Y_train, self.Y_test = self.train_test_data
    DT_model_reg = DecisionTreeRegressor(random_state = 0, max_depth = 5)
    DT_model_reg.fit(X_train, Y_train)
    Y_pred = DT_model_reg.predict(X_test)
    print("[decision_tree]")
    print("accuracy : {:.3f}".format(DT_model_reg.score(X_train, Y_train)))
    rmse = sqrt(mean_squared_error(self.Y_test, Y_pred))
    print("rmse :  {:.3f}".format(rmse))
    self.Y_value = [Y_pred, self.Y_test]
    
    return[X_train, DT_model_reg]

  def SVR_function(self, train_test_data):
   X_train, X_test, Y_train, self.Y_test = self.train_test_data
   SVR_model = SVR(C=1, kernel = "linear", epsilon = 0.1)
   SVR_model.fit(X_train, Y_train)
   Y_pred = SVR_model.predict(X_test)
   print("[SVM : Support Vector Machine]")
   print("accuracy : {:.3f}".format(SVR_model.score(X_train, Y_train)))
   rmse = sqrt(mean_squared_error(self.Y_test, Y_pred))
   print("RMSE : {:.3f}".format(rmse))
   print(SVR_model.coef_)
   self.Y_value = [Y_pred, self.Y_test]

  def NaiveBayes(self, train_test_data):
    X_train, X_test, Y_train, self.Y_test = self.train_test_data
    NB_model_reg = BayesianRidge(alpha_1 = 0.001, lambda_1 = 0.001)
    NB_model_reg.fit(X_train, Y_train)
    Y_pred = NB_model_reg.predict(X_test)
    print("accuracy : {:.3f}".format(NB_model_reg.score(X_train, Y_train)))
    rmse = sqrt(mean_squared_error(self.Y_test, Y_pred))
    print("rmse :  {:.3f}".format(rmse))
    print(NB_model_reg.coef_)
    self.Y_value = [Y_pred, self.Y_test]

  def MLP(self, train_test_data):
    X_train, X_test, Y_train, self.Y_test = self.train_test_data
    nn_reg_model = MLPRegressor(random_state = 0, alpha = 0.0001, max_iter = 2000, 
                            hidden_layer_sizes = [50, 50])
    nn_reg_model.fit(X_train, Y_train)
    Y_pred = nn_reg_model.predict(X_test)
    print("Y predict value : \n", Y_pred)
    print("train accuracy : {:.3f}".format(nn_reg_model.score(X_train, Y_train)))
    
    rmse = sqrt(mean_squared_error(self.Y_test, Y_pred))
    print("RMSE : {:.3f}".format(rmse))
    self.Y_value = [Y_pred, self.Y_test]

  def DNN(self,train_test_data):
      np.random.seed(0)
      tf.random.set_seed(0)

      X_train, X_test, Y_train, self.Y_test = self.train_test_data

      X_train2 = pd.DataFrame(X_train)
      model = keras.models.Sequential()
      model.add(keras.layers.Dense(64, input_dim=len(X_train2.columns), activation = "relu"))
      model.add(keras.layers.Dense(64, activation = "relu"))
      model.add(keras.layers.Dense(64, activation = "relu"))

      #6. 모형 학습
      model.compile(loss = "mse", optimizer = "SGD")
      Y_pred = np.round(model.predict(X_test[:5], verbose=0), 3)
      print("Y predict value \n", Y_pred)
      #7. 모형 평가
      train_score = model.evaluate(X_train, Y_train, verbose=0)
      test_score = model.evaluate(X_test, self.Y_test, verbose=0)
      print("train mse : {:.3f}".format(train_score))
      print("test mse : {:.3f}".format(test_score))
      self.Y_value = [Y_pred, self.Y_test]

      #print("train accuracy : {:.3f}".format(train_score[0], train_score[1]))
      #print("test accuracy : {:.3f}".format(test_score[0], test_score[1]))

  def RandomForest(self,train_test_data):
    X_train, X_test, Y_train, self.Y_test = self.train_test_data
    model = RandomForestRegressor(random_state = 0, n_estimators = 100, max_depth = 4)
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    print("Y predict value : \n", Y_pred)
    print("accracy(R2) : {:.3f}".format(model.score(X_train, Y_train)))
    rmse = sqrt(mean_squared_error(self.Y_test, Y_pred))
    print("RMSE :", rmse)
    self.Y_value = [Y_pred, self.Y_test]

  def Gradien(self,train_test_data):
    X_train, X_test, Y_train, self.Y_test = self.train_test_data
    model = GradientBoostingRegressor(random_state = 0, n_estimators = 100, max_depth = 4, 
                                   learning_rate = 0.1)
    #5. 모형 학습 예측
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    print("Y predict value : \n", Y_pred)
    print("accracy(R2) : {:.3f}".format(model.score(X_train, Y_train)))
    rmse = sqrt(mean_squared_error(self.Y_test, Y_pred))
    print("RMSE :", rmse)
    self.Y_value = [Y_pred, self.Y_test]
    return model
